In [ ]:
def simplex(matrix, rhs, z, direction=1):
    '''Simplex algorithm to solve linear programming problems
    
    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side
    
    rhs: numpy ndarray
        Right-hand side vector
    
    numxvars: int
        Number of x variables
        
    direction: {+1 , -1}
        For maximization problems use +1 and for minimization problems use -1 instead.
    
    Returns
    ---------
    solutions: np.ndarray
        Array of solutions for every iteration when an optimal solution was found
    favlues: np.ndarray
        Array of the objective function values in every iteration
    lastrows: np.ndarray
        Array of the last two rows (zj, cj - zj) in the optimal solution table
    ''' 
    matrix = np.array(matrix, dtype=float)
    rhs = np.array(rhs)
    z = np.array(z)
    
    
    num_rows, num_cols = matrix.shape
    labels = [f"x{i + 1}" for i in range(num_cols)]
    
    cb_index = np.where((matrix == 1) & (np.abs(matrix).sum(axis=0) == 1))[1]
    cb = z[cb_index]
                       
    zj = cb.dot(matrix)
                       
    net_evaluation = direction * (z - zj)
    
    solutions = []
    fvalues = []
    cbindexlist = []
    iteration = 0
    while np.any(net_evaluation > 0):
        solution = np.zeros_like(z)
        entering = net_evaluation.argmax()  # entering variables (index)
        entering_label = labels[entering]

        key_col = matrix[ : , entering]
        ratios = np.divide(rhs, key_col, out=np.full_like(rhs, np.inf), where=key_col>0)
        leaving = ratios.argmin()   # leaving variables (index)
        leaving_label = labels[cb_index[leaving]]     
        
        pivot = matrix[leaving, entering]
        
        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot
        
        for i in range(num_rows):
            if i == leaving:
                continue
            factor = matrix[i, entering]
            matrix[i] = -factor * matrix[leaving] + matrix[i]
            rhs[i] = -factor * rhs[leaving] + rhs[i]
        
        cb_index[leaving] = entering
        cb = z[cb_index]
        
        zj = cb.dot(matrix)
        
        net_evaluation = direction * (z - zj)

        solution[cb_index] = rhs  # basics
        
        iteration += 1
        print(f"Iteration {iteration}.  {leaving_label} --> {entering_label}")
        print(matrix,  "\n")
        print("cb", cb, "\n")
        print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}", "\n")
        
        solutions.append(solution)
        fvalues.append(cb.dot(rhs))
        cbindexlist.append(cb_index)
        if np.all(net_evaluation <= 0):
            print(f"Optimal solution found in {iteration} iterations")
    return np.array(solutions), fvalues, np.vstack((zj, net_evaluation)), np.array(cbindexlist)